In [ ]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
display(HTML("<style>.container {width:100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [ ]:
%load_ext autoreload
%autoreload 2
    
import bnpm

In [ ]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc
import time

------------
# ==== Start ====

## Prepare username + password

In [ ]:
# remote_host_transfer = "transfer.rc.hms.harvard.edu"
# remote_host_compute = "o2.hms.harvard.edu"

remote_host_compute = "rc.fas.harvard.edu"

username = input('Username: ')

## Prepare `sftp` and `ssh` objects

In [ ]:
## initialize ssh_transfer
if remote_host_compute == 'rc.fas.harvard.edu':
    ssh_c = bnpm.server.ssh_interface()
    ssh_c.fasrc_connect(username=username)
    ssh_t = ssh_c
elif remote_host_compute == 'o2.hms.harvard.edu':
    use_localSshKey = False
    pw = bnpm.server.pw_encode(getpass.getpass(prompt='Password: ')) if (use_localSshKey==False) else None
    path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None
    
    ssh_t = bnpm.server.ssh_interface(
        nbytes_toReceive=20000,
        recv_timeout=1,
        verbose=True,
    )
    ssh_t.o2_connect(
        hostname=remote_host_transfer,
        username=username,
        password=bnpm.server.pw_decode(pw),
        key_filename=path_sshKey,
        look_for_keys=False,
        passcode_method=1,
        verbose=0,
        skip_passcode=False,    
    )
    ## initialize ssh_compute
    ssh_c = bnpm.server.ssh_interface(
        nbytes_toReceive=20000,
        recv_timeout=1,
        verbose=True,
    )
    ssh_c.o2_connect(
        hostname=remote_host_compute,
        username=username,
        password=bnpm.server.pw_decode(pw),
        key_filename=path_sshKey,
        look_for_keys=False,
        passcode_method=1,
        verbose=0,
        skip_passcode=False,    
    )

In [ ]:
sftp = bnpm.server.sftp_interface(ssh_client=ssh_t.client)

prompt_snip = f'[{username}'
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

## Find files and directories with data

In [ ]:
paths_found = sftp.search_recursive(
    # dir_outer='/n/data1/hms/neurobio/sabatini/rich/data/2pRAM/facerhythm_stroke_biomarker_exp/camera/20250303/PS46',
    dir_outer='/n/netscratch/bsabatini_lab/Lab/rhakim/data/2pRAM/',
    reMatch='video1.*avi',
    # reMatch_in_path='20250303',
    depth=5,
    find_folders=False,
    find_files=True,
    verbose=True,
)

In [ ]:
import natsort

In [ ]:
import bnpm.path_helpers


dates_toUse = natsort.natsorted([bnpm.path_helpers.find_date_in_path(path).replace('-', '') for path in paths_found])
dates_toUse

In [ ]:
# dates_toUse = [
#     '20231022',
#     '20231023',
#     '20231024',
#     '20231025',
#     '20231026',
#     '20231027',
#     '20231028',
#     '20231029',
#     '20231030',
#     '20231031',
#     '20231101',
#     '20231102',
#     '20231103',
#     '20231104',
#     '20231105',
#     '20231106'
# ]

## Start an interactive job for dispatching

In [ ]:
# send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=2G /bin/bash');
send_expect(f'salloc -p kempner --account=kempner_bsabatini_lab --mem=80000 --gres=gpu:1 -c 2 -N 1 -t 0-08:00');

## Pull/update repo

In [ ]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
send_expect('git pull');

## Activate environment

In [ ]:
# name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'
name_env = '/n/home03/rhakim/.conda/envs/fr'

send_expect(f'conda activate {name_env}');

## Install face-rhythm

In [ ]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
send_expect(f'pip install -e .[all_cv2Headless] --upgrade')

In [ ]:
# names_st = ['rh183']*0 + ['joz608']*0 + ['km335']*0 + ['gyh930']*0 + ['ccb18']*0 + ['pac4988']*0 + ['aeg38']*0 + ['shl143']*1
# for date, name_st in zip(dates_toUse, names_st):
for date in dates_toUse[:]:
    # cage = 'cage_0916N'
    mouse = 'PS46'

    name_slurm = mouse + '_' + date[-4:]
    name_job = 'jobNum_'
    
    # dir_videos       = str(Path('/n/data1/hms/neurobio/sabatini/rich/data/2pRAM/facerhythm_stroke_biomarker_exp/camera/') / date / mouse )
    # dir_ROIs         = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / f'{date}' / 'run_20250308' / 'ROIs_prealigned' / 'ROIs.h5')
    # dir_saveOutputs  = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20250309' / date)
    # path_script_remote = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    dir_videos         = str(Path('/n/netscratch/bsabatini_lab/Lab/rhakim/data/2pRAM/facerhythm_stroke_biomarker_exp/camera/') / mouse / date )
    dir_ROIs           = str(Path('/n/holylabs/LABS/bsabatini_lab/Users/rhakim/analysis/face_rhythm') / mouse / f'{date}' / 'run_20250308' / 'ROIs_prealigned' / 'ROIs.h5')
    dir_saveOutputs    = str(Path('/n/holylabs/LABS/bsabatini_lab/Users/rhakim/analysis/face_rhythm') / mouse / 'run_20250320' / date)
    path_script_remote = str(Path(r'/n/holylabs/LABS/bsabatini_lab/Users/rhakim/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server


    path_dispatcher_local   = str(Path(r'/Users/richardhakim/Documents/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server



    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
        'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_videos} \
    {dir_ROIs} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    ## Upload `dispatcher.py` file
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));


    send_expect(commands['dispatch']);


    ## For serverteeny usage
#     send_expect(f"chmod 777 {path_dispatcher_remote}")
#     run_file = {
#         "name": "Rich",
#         "o2_acct": str(name_st),
#         "notes": f"basic_run_{mouse}_{date}",
#         "command": f"sbatch /n/data1/hms/neurobio/sabatini/rich/serverteeny_submitter.sh {commands['dispatch']}",
#     }
#     path_temp = str(Path(tempfile.gettempdir()) / f'run_{date}.json')
#     bnpm.file_helpers.json_save(
#         obj=run_file,
#         filepath=path_temp,
#     )
#     sftp.sftp.put(
#         localpath=path_temp,
#         remotepath=str(Path('/n/data1/hms/neurobio/sabatini/serverteeny/run') / Path(path_temp).name),
#     );
        

In [ ]:
ssh_c.expect()

## Prepare directories

In [ ]:
import numpy as np

mouse = 'mouse_00916N'
date = '20230420'

name_slurm = date[-4:]

dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments') / mouse / 'camera_data' / date)

dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'ROIs_prealigned' / f'{mouse}__{date}' / 'ROIs.h5')

dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / date)


name_job = 'jobNum_'


path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server


## Prepare commands

In [ ]:
## Expectation for dispatch args: 
# path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env

commands = {
#     'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
#     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
python3 \
{path_dispatcher_remote} \
{path_script_remote} \
{dir_saveOutputs} \
{dir_videos} \
{dir_ROIs} \
{name_job} \
{name_slurm} \
{name_env}",
}
display(commands)

## Upload `dispatcher.py` file

In [ ]:
sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch

In [ ]:
send_expect(commands['dispatch']);

In [ ]:
# for date in dates_toUse:
for date in dates_toUse[:]:
    # mouse = 'mouse_0403L'
    mouse = 'mouse_0322N'
    # date = '20230420'
    
    name_slurm = date[-4:]
    
    dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments') / mouse / 'camera_data' / date)
    
    # dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230903' / 'ROIs_prealigned' / f'{date}' / 'ROIs.h5')
    dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'ROIs_prealigned' / f'{mouse}__{date}' / 'ROIs.h5')
    
    # dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230903' / date)
    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / date)
    
    
    
    name_job = 'jobNum_'
    
    
    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server
    
    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server
    
    
    ## Prepare commands
    
    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env
    
    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
        'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_videos} \
    {dir_ROIs} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    ## Upload `dispatcher.py` file
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    ## Dispatch
    
    # send_expect(commands['dispatch']);

    # time.sleep(60)

In [ ]:
paths_all = sftp.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/',
    search_pattern_re='VQT_Analyzer.h5',
    max_depth=8,
)

In [ ]:
{p: f['last_modified'] for p, f in sftp.get_fileProperties(paths_all, error_on_missing=True).items()}

In [ ]:
send_expect('pip uninstall face-rhythm')

In [ ]:
send_expect('y')

In [ ]:
send_expect('pip install -e .')

In [ ]:
send_expect('pip list')

In [ ]:
send_expect()

## Run with serverteeny

### check on job

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()